In [12]:
from lerobot.common.datasets.lerobot_dataset import LeRobotDataset
from pathlib import Path
import os
import torch
import numpy as np

Load the dataset that you want to clean up by removing all the zero actions

In [13]:
root_dataset_dir = '/fast_storage/qnoens/OpenVLA/data/lerobot_blue'
dataset_name = 'test_dataset'

lerobot_dataset = LeRobotDataset(repo_id=dataset_name, root=root_dataset_dir)
meta = lerobot_dataset.meta
print(meta)

Resolving data files:   0%|          | 0/150 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

LeRobotDatasetMetadata({
    Repository ID: 'test_dataset',
    Total episodes: '150',
    Total frames: '20211',
    Features: '['next.reward', 'next.success', 'seed', 'timestamp', 'wrist_image_original', 'scene_image_original', 'wrist_image', 'scene_image', 'state', 'robot_pose', 'gripper_state', 'joints', 'action', 'frame_index', 'episode_index', 'index', 'task_index']',
})',



Decide on the path where you want the new dataset to end up

In [14]:
root_dir = Path('/fast_storage/qnoens/OpenVLA/data/lerobot_blue_only_zero_actions2')
use_videos = Path(root_dataset_dir + '/' + 'videos').exists()

#### Create the new dataset. 

**Note:** If the path you gave already existed, then it will deleted!

In [18]:
if root_dir.exists():
    print(f'Root directory already exists. Deleting it.')
    os.system(f'rm -rf {root_dir}')

new_dataset = LeRobotDataset.create(
    repo_id=dataset_name,
    fps=lerobot_dataset.fps,
    root=root_dir,
    features=lerobot_dataset.features,
    use_videos=use_videos,
    image_writer_processes=0,
    image_writer_threads=16,
)

Root directory already exists. Deleting it.


The `episodes_to_delete` list enables us to exclude any unwanted episodes from the original dataset and therefore won't be added to the new one.

In [19]:
episodes_to_delete = []

We will now go episode by episode over every frame within the episode to create the new episodes excluding actions that are too small and will therefore damage the performance of our model.

**Note:** The relative gripper actions are also made binary here which makes it immediately compatible with OpenVLA

In [ ]:
# We now go episode by episode over frame by frame
episode_count = 0
gripper_threshold = 0.06

for ep_idx in range(lerobot_dataset.num_episodes):
    if ep_idx in episodes_to_delete:
        print(f"Skipping episode {ep_idx} because it is deleted.")
        continue
    episode_meta = meta.episodes[ep_idx]    
    episode_length = episode_meta['length']
    #instruction = episode_meta['tasks'][0]
    instruction = "move the block to the blue rectangle"
    lengths = [meta.episodes[i]['length'] for i in range(ep_idx)]
    start_idx = sum(lengths)

    first_gripper_state = lerobot_dataset[start_idx]['gripper_state'].numpy()
    prev_gripper_state = 0.0 if first_gripper_state < gripper_threshold else 1.0

    # Iterate over frames and set action to zero
    remover_count = 0
    for frame_idx in range(start_idx, start_idx + episode_length):
        orig_frame = lerobot_dataset[frame_idx]

        if ep_idx >= 100:
            # Check if action is zero
            action = orig_frame['action'].numpy()
            if np.all(np.isclose(action, 0.0, atol=2e-3)):
                remover_count += 1
                continue

            # Logic to remove zero gripper actions, only possible if [x,y,z,rx,ry,rz] are zero
            # This also immediately sets the gripper action to binary values (for OpenVLA)
            if np.all(np.isclose(action[:6], 0.0, atol=2e-3)):
                gripper_action = action[6]
                if gripper_action < 0.0 and prev_gripper_state == 1.0: # Closing
                    prev_gripper_state = 0.0
                    action[6] = 0.0
                    orig_frame['action'] = torch.from_numpy(action)
                    print(f"Frame {frame_idx} gripper state changed to 0.0")
                elif gripper_action > 0.0 and prev_gripper_state == 0.0: # Opening
                    prev_gripper_state = 1.0
                    action[6] = 1.0
                    orig_frame['action'] = torch.from_numpy(action)
                    print(f"Frame {frame_idx} gripper state changed to 1.0")
                else:
                    remover_count += 1
                    continue
            else:
                action[6] = prev_gripper_state
                orig_frame['action'] = torch.from_numpy(action)
        
        # Create new frame
        frame = {
            'next.reward': orig_frame['next.reward'].unsqueeze(0),
            'next.success': orig_frame['next.success'].unsqueeze(0),
            'seed': orig_frame['seed'].unsqueeze(0),  
            'wrist_image_original': orig_frame['wrist_image_original'].permute(1, 2, 0),
            'scene_image_original': orig_frame['scene_image_original'].permute(1, 2, 0),
            'wrist_image': orig_frame['wrist_image'].permute(1, 2, 0),
            'scene_image': orig_frame['scene_image'].permute(1, 2, 0),
            'state': orig_frame['state'], 
            'robot_pose': orig_frame['robot_pose'], 
            'gripper_state': orig_frame['gripper_state'].unsqueeze(0), 
            'joints': orig_frame['joints'], 
            'action': orig_frame['action'],
            'task': instruction
        }

        # Add frame to new dataset
        new_dataset.add_frame(frame)


    # Add episode to new dataset
    episode_count += 1
    print(f'Removed {remover_count} frames from episode {episode_count}/{lerobot_dataset.num_episodes - len(episodes_to_delete)}')
    new_dataset.save_episode()




Removed 0 frames from episode 1/150


Map:   0%|          | 0/64 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Removed 0 frames from episode 2/150


Map:   0%|          | 0/130 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/97 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Frame 20144 gripper state changed to 0.0
Frame 20184 gripper state changed to 1.0
Removed 36 frames from episode 150/150


Map:   0%|          | 0/104 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]